In [2]:
import channel as CH
import numpy as np
import matplotlib.pyplot as plt

if __name__ == '__main__':
    ch = CH.CHANNEL.from_toml('../case-run/case-debug/case.toml')
    for i in range(1):
        id = 10 * i + 30
        print(id)
        ch.add_var_frdir(f'../case-run/case-debug/DATA/F-{id:08d}.H5')

        ch.add_var('velx', CH.s2p(ch.get_var('velx_fft')))
        ch.add_var('vely', CH.s2p(ch.get_var('vely_fft')))
        ch.add_var('velz', CH.s2p(ch.get_var('velz_fft')))

        ch.add_var('vorx', CH.s2p(ch.get_var('vorx_fft')))
        ch.add_var('vory', CH.s2p(ch.get_var('vory_fft')))
        ch.add_var('vorz', CH.s2p(ch.get_var('vorz_fft')))

        ch.add_var('lmbx', CH.s2p(ch.get_var('lmbx_fft')))
        ch.add_var('lmby', CH.s2p(ch.get_var('lmby_fft')))
        ch.add_var('lmbz', CH.s2p(ch.get_var('lmbz_fft')))

        ch.write_vtk(f'../case-run/case-debug/VTK/F-{id:08d}.vtk', [
            'velx', 'vely', 'velz', 'vorx', 'vory', 'vorz', 'lmbx', 'lmby',
            'lmbz'
        ])

30


In [ ]:
u_mean = np.mean(ch.get_var('velx'), axis=(0, 1))
u = ch.get_var('velx') - u_mean[np.newaxis, np.newaxis, :]
uu = np.mean(np.power(u, 2), axis=(0, 1))

yplus = 180 * (1 - np.abs(1 - ch.y))
utau = 180 / 2800
fig, ax = plt.subplots()
ax.plot(yplus, u_mean / utau)
ax.set(xlim=(1.e-1, 200), ylim=(0, 20))
ax.set_xscale('log')
ax.grid('on')

fig, ax = plt.subplots()
ax.plot(yplus, uu / utau**2)
ax.set(xlim=(1.e0, 200), ylim=(0, 8))
ax.set_xscale('log')
ax.grid('on')

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots()
u_fft = np.abs(ch.get_var('velx_fft')[:, :, 6])
u_fft[0, 0] = 0
ax.matshow(u_fft / u_fft.max())

In [ ]:
import matplotlib.pyplot as plt

# print(ch.D1[0, :])
# print(ch.D1[-1, :])
fig, ax = plt.subplots()
ax.plot(ch.get_var('vory_fft')[2, 4, :], '.-')
# ax.plot(ch.get_var('vorz')[0, 0, :])
ax.plot(-ch.D1 @ (ch.get_var('vory_fft')[2, 4, :]))

# ax.plot()

# ax.set(xlim=(105, 112), ylim=(0, 1))

In [ ]:
import channel as CH
import numpy as np


def is_postive_definite(mat):
    try:
        np.linalg.cholesky(mat)
        return True
    except np.linalg.LinAlgError:
        return False


if __name__ == '__main__':
    ch = CH.CHANNEL.from_toml('../case-run/case.toml')
    gamma_0 = 11.0 / 6.0
    dt = 1.e-3
    k2 = 128
    k4 = k2**2
    A = -ch.nu * ch.DY2 + (gamma_0 / dt + ch.nu * k2) * np.eye(ch.Ny)

    # BC
    A[0, :] = 0
    A[0, 0] = 1
    A[ch.Ny - 1, :] = 0
    A[ch.Ny - 1, ch.Ny - 1] = 1

    B = -ch.nu * ch.DY4 + (gamma_0 / dt + 2 * ch.nu * k2) * ch.DY2 - (
        gamma_0 * k2 / dt + ch.nu * k4) * np.eye(ch.Ny)
    # BC
    B[0, :] = 0
    B[0, 0] = 1
    B[1, :] = ch.DY1[0, :]
    B[ch.Ny - 2, :] = ch.DY1[ch.Ny - 1, :]
    B[ch.Ny - 1, :] = 0
    B[ch.Ny - 1, ch.Ny - 1] = 1

    # check matrix is postive defined
    print(is_postive_definite(A))
    print(np.linalg.matrix_rank(B))


In [ ]:
import channel as CH
import numpy as np
import os
import sys
import matplotlib.pyplot as plt

# check mean velocity solver

if __name__ == '__main__':
    ch = CH.CHANNEL.from_toml('../case-run/case.toml')
    id = 0
    dt = 1.e-3
    ch.add_var_frdir(f'../case-run/DATA/F-{id:08d}.H5')

    print(ch.get_var('lmbx_fft')[:, :, 32])
    u0 = ch.get_var('velx_fft')[0, 0, :]
    w0 = ch.get_var('velz_fft')[0, 0, :]

    fx_fric_0 = 3 / 2800
    print(fx_fric_0)

    alpha_0 = 3.0
    alpha_1 = -1.5
    alpha_2 = 1.0 / 3.0
    beta_0 = 3.0
    beta_1 = -3.0
    beta_2 = 1.0
    gamma_0 = 11.0 / 6.0

    source_u = (beta_0 + beta_1 + beta_2) * (fx_fric_0 +
                                             ch.get_var('lmbx_fft')[0, 0, :])
    source_u += (alpha_0 + alpha_1 + alpha_2) * u0 / dt

    source_w = (beta_0 + beta_1 + beta_2) * ch.get_var('lmbz_fft')[0, 0, :]
    source_w += (alpha_0 + alpha_1 + alpha_2) * w0 / dt

    print(source_w)
    A = -ch.nu * ch.DY2 + gamma_0 / dt * np.eye(ch.Ny)

    # BC
    A[0, :] = 0
    A[0, 0] = 1
    A[ch.Ny - 1, :] = 0
    A[ch.Ny - 1, ch.Ny - 1] = 1
    # print(A)

    source_u[0] = 0
    source_u[-1] = 0
    # print(source_u)
    u = np.linalg.solve(A, source_u)

    # print(u)
    fig, ax = plt.subplots()
    ax.plot(np.real(u), ch.y)
    ax.plot(np.real(u0), ch.y)

In [ ]:
# NX2 = NX * 3 // 2
# NZ2 = NZ * 3 // 2
# x2 = np.arange(NX2) * 4 * np.pi / (NX2)
# z2 = np.arange(NZ2) * 2 * np.pi / (NZ2)
# write_vtk(
#     [velx_p, vely_p, velz_p, vorx_p, vory_p, vorz_p, lmbx_p, lmby_p, lmbz_p],
#     ['velx', 'vely', 'velz', 'vorx', 'vory', 'vorz', 'lmbx', 'lmby', 'lmbz'],
#     x2, y, z2, 'initial_refined_field.vtk')

# plt.show()

In [ ]:
import channel as CH
import numpy as np
import os
import sys
import matplotlib.pyplot as plt

# check fluctuating velocity solver

if __name__ == '__main__':
    ch = CH.CHANNEL.from_toml('../case-run/case.toml')
    id = 0
    dt = 1.e-3
    ch.add_var_frdir(f'../case-run/DATA/F-{id:08d}.H5')
    kxid = 4
    kzid = 2
    kx = ch.kx[kxid]
    kz = ch.kz[kzid + ch.NzH]
    k2 = np.power(kx, 2) + np.power(kz, 2)
    lmbx = ch.get_var('lmbx_fft')[kzid, kxid, :]
    lmbz = ch.get_var('lmbz_fft')[kzid, kxid, :]
    vory = ch.get_var('vory_fft')[kzid, kxid, :]
    vely = ch.get_var('vely_fft')[kzid, kxid, :]
    Hg = 1j * kz * lmbx - 1j * kx * lmbz

    alpha_0 = 3.0
    alpha_1 = -1.5
    alpha_2 = 1.0 / 3.0
    beta_0 = 3.0
    beta_1 = -3.0
    beta_2 = 1.0
    gamma_0 = 11.0 / 6.0

    # calculate source_g
    source_g = (beta_0 + beta_1 + beta_2) * Hg + (alpha_0 + alpha_1 +
                                                  alpha_2) * vory / dt
    # calculate Hg
    print(f'kx {kx:.1f} kz {kz:.1f}')
    fig, ax = plt.subplots()
    ax.plot(np.real(lmbz), ch.y, label='real')
    ax.plot(np.imag(lmbz), ch.y, label='imag')
    ax.legend()

In [ ]:
import numpy as np
from scipy.fft import fft2, fftfreq
import matplotlib.pyplot as plt

NX = 48
NZ = 32
LX = 4 * np.pi
LZ = 2 * np.pi
dx = LX / NX
dz = LZ / NZ
X, Z = np.meshgrid(np.arange(NX) * dx, np.arange(NZ) * dz)
k1 = 2.0
k2 = 1.0
u = np.cos(k1 * X) * np.sin(k2 * Z)

# FFT
u_fft = fft2(u, norm='forward')
kx = fftfreq(NX, d=dx / (2 * np.pi))
kz = fftfreq(NZ, d=dz / (2 * np.pi))

print(kz[NZ - 1], kx[4], u_fft[NZ - 1, 4])

fig, ax = plt.subplots()
# ax.contourf(X, Z, u)
ax.matshow(np.abs(u_fft))

In [ ]:
# this cell checks the 2nd-order ODE
import numpy as np
import channel as CH
import matplotlib.pyplot as plt


def coth(x):
    return np.cosh(x) / np.sinh(x)


def sol_homo(lambda_, k_, h_, y):
    f = np.cosh(lambda_ * y) - np.cosh(k_ * y) - coth(lambda_ * h_) * np.sinh(
        lambda_ * y) + coth(k_ * h_) * np.sinh(k_ * y)
    f /= (lambda_**2 - k_**2)
    return f


ch = CH.CHANNEL.from_toml('../case-run/case.toml')

k = 2.0
k2 = np.power(k, 2)
gamma_0 = 11.0 / 6.0
dt = 1.e-1
print(1 / ch.nu)
lambda2 = k2 + gamma_0 / (dt * ch.nu)
A1 = -ch.DY2 + lambda2 * np.eye(ch.Ny)

# solve phi
A1[0, :] = 0
A1[0, 0] = 1
A1[-1, :] = 0
A1[-1, -1] = 1

source_phi = np.zeros(ch.Ny)
source_phi[0] = 1

phi = np.linalg.solve(A1, source_phi)
# print(phi)

A2 = ch.DY2 - k2 * np.eye(ch.Ny)
A2[0, :] = 0
A2[0, 0] = 1
A2[-1, :] = 0
A2[-1, -1] = 1

# np.linalg.cholesky(A2)
source_v = np.copy(phi)
source_v[0] = 0
source_v[-1] = 0
v = np.linalg.solve(A2, source_v)

fig, ax = plt.subplots()
# ax.plot(ch.y, phi, label='phi')
ax.plot(ch.y, v, label='v')
ax.plot(ch.y, sol_homo(np.sqrt(lambda2), k, 2, ch.y), label='v true')
ax.legend()

# print(v)
# ax.set_yscale('log')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

N = 192
eta = np.linspace(-1, 1, N)

ap = 1.8
aeta = ap * eta
ap2 = np.power(ap, 2)
ap3 = np.power(ap, 3)
ap4 = np.power(ap, 4)
tap = np.tanh(ap)

y = np.tanh(aeta) / tap
sech2 = np.power(np.cosh(aeta), -2)
tanh2 = np.power(np.tanh(aeta), 2)
taeta = np.tanh(aeta)

dyde1 = ap / np.tanh(ap) * np.power(np.cosh(aeta), -2)
dyde2 = -2 * ap2 / np.tanh(ap) * np.power(np.cosh(aeta), -2) * np.tanh(aeta)
dyde3 = (-2 * ap3 * sech2**2 + 4 * ap3 * sech2 * tanh2) / tap
dyde4 = (16 * ap4 * sech2**2 * taeta - 8 * ap4 * sech2 * taeta**3) / tap

fig, ax = plt.subplots()
ax.plot(eta, y)
ax.plot(eta, dyde1, label='1st')
ax.plot(eta, dyde2, label='2nd')
ax.plot(eta, dyde3, label='3rd')
ax.plot(eta, dyde4, label='4th')
ax.legend()